# Make tables: calculate p-values and differential expressions for each cancer 

Create a df with FDR p-value results for proteins in the MAPK signaling pathway (t-test of trans phosphoproteomics with KRAS hotspot compared to KRAS wildtype). The df also includes differential expressions. The three cancers with frequent hotspot mutations are included in the df (EC, CO, LUAD).

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re

import cptac
import cptac.utils as u
import plot_utils as p

In [2]:
print('cptac version:', cptac.version())

cptac version: 0.9.6


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
'''
Params
cancer_object: Object of the loaded cancer data set.
all_prot: List of trans proteins used to get phosphoproteomics data. 
gene_in: String. Gene used to get mutation status for the Mutation col.
utils: utils package from cptac.  

Returns a dataframe with trans phosphoproteomics and mutation status of a specific gene.
'''

def phospho_format_df(cancer_object, all_prot, gene_in = 'KRAS', utils = u):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    mut_type['Mutation'].where(mut_type['Mutation'] != 'Missense_Mutation', 'Missense', inplace = True) # replace when false
    mut_type['Mutation'].where(mut_type['Mutation'] != 'nonsynonymous SNV', 'Missense', inplace = True)

    # Keep only tumor samples from proteomics
    prot_and_mutations = cancer_object.join_omics_to_mutations(
        mutations_genes = [gene_in], omics_df_name = 'phosphoproteomics', omics_genes = all_prot, tissue_type = 'tumor')
    # Reduce multiindex keeping phosphosite
    if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
        if cancer_object.get_cancer_type() in ('endometrial', 'colon'):
            prot_and_mutations = u.reduce_multiindex(prot_and_mutations, flatten=True) 
        elif cancer_object.get_cancer_type() in ('luad'):
            prot_and_mutations = u.reduce_multiindex(prot_and_mutations, levels_to_drop = [2,3], flatten = True) 
        prot_and_mutations = rename_duplicate_cols(prot_and_mutations) # make unique cols

    # Keep certain missense mutations
    prot_and_mutations[gene_in+'_Location'] = [','.join(map(str, l)) for l in prot_and_mutations[gene_in+'_Location']]
    hotspots = ['G12', 'G13', 'Q61', 'No_mutation']
    hotspots_wt = pd.DataFrame()
    for site in hotspots:
        df = prot_and_mutations[prot_and_mutations.KRAS_Location.str.contains(site, regex= True, na=False)]
        hotspots_wt = hotspots_wt.append(df)
    #print(hotspots_wt.KRAS_Location.value_counts())
    
    # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
    mut_type = mut_type[['Mutation']]
    prot_df = hotspots_wt.iloc[:,:-4] # Keep only proteomics
    merged = prot_df.join(mut_type)
    merged = merged.join(hotspots_wt[gene_in+'_Location'])

    # Keep only Wildtype and deletion
    compare = ['Wildtype_Tumor','Missense']
    get = merged['Mutation'].isin(compare)
    mut_wt = merged[get]
    
    mut_wt = mut_wt.rename(columns={col: re.sub(r'_phosphoproteomics', '', col) for col in mut_wt.columns.tolist()})
    return mut_wt


In [5]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Formatted df with samples with hotspot missense or wt for KRAS.
all_prot_list: List. All proteins in proteomics data frame. 

Returns a df with differential expression for phosphoproteomics (KRAS hotspot - wt). 
A pos value represents an increase in phosphorylation abundance with mutant KRAS. 
A neg value represents an decrease in phosphorylation abundance with mutant KRAS.
'''

def get_change_in_medians_df(cancer_object, cancer_name, mut_wt_df, all_prot_list):
    miss = mut_wt_df[mut_wt_df.Mutation == "Missense"]
    wt = mut_wt_df[mut_wt_df.Mutation == "Wildtype_Tumor"]
    miss_med = miss.median()
    wt_med = wt.median()

    med_dict = {}
    
    for prot in all_prot_list:
        if prot in miss_med.index and prot in wt_med.index:
            dif = miss_med[prot] - wt_med[prot]
            med_dict[prot] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Phospho'})
    
    return df
        

In [6]:
''' Adds '_i' to duplicate col names where the int i increases by one with each duplicate
Returns a df with unique column names. '''

def rename_duplicate_cols(df):
    cols = pd.Series(df.columns[:])

    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

    # rename the columns with the cols list.
    df.columns=cols
    return df

In [7]:
'''
df: DataFrame containing omics data and a binary column.
mincount: Int of the minimum num of actual values needed to pass cutoff and not be listed in not_enough_data list. 
omics_cols: Names of columns to check if there is enough data. 

Returns: List of genes with not enough data (num of non-NaN values are less than mincount).
'''

def find_few_data_genes(df, mincount, omics_cols = None, binary_labels = ('Missense','Wildtype_Tumor'), 
    binary_col = 'Mutation', gene = 'KRAS'):
    
    # Separate into binary groups
    label_1 = binary_labels[0]
    label_2 = binary_labels[1]
    partition1 = df.loc[df[binary_col] == label_1]
    partition2 = df.loc[df[binary_col] == label_2]
    
    # Get list of columns
    if omics_cols is None:
        omics_cols = list(df.columns)
        omics_cols.remove(binary_col)
    else:
        set_omics_cols = set(omics_cols)
        if binary_col in (set_omics_cols):
            omics_cols.remove(binary_col)
        
    # Append genes with less data than mincount (for mut or wt) to list
    not_enough_data = []
    for c in omics_cols:
        #print(len(partition1[c].dropna(axis='rows')))
        #print(len(partition2[c].dropna(axis='rows')))
        if len(partition1[c].dropna(axis='rows')) <= mincount: #non-nan vals less than min
            not_enough_data.append(c)
            continue
        elif len(partition2[c].dropna(axis='rows')) <= mincount:
            not_enough_data.append(c)
            continue
        
    
    print('Num genes with not enough data: ', len(not_enough_data), '/', len(omics_cols))        
    return not_enough_data

#  Step 1: Create data frames with p-values and differential expressions

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run t-tests for all genes in the proteomic data frame. Use get_change_in_medians_df to create the data frame with differential expression values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [8]:
en = cptac.Endometrial()
l = cptac.Luad()
col = cptac.Colon()

In [9]:
# Get MAPK pathway proteins
prot = u.get_proteins_in_pathways('MAPK Signaling Pathway', 'wikipathways')
prot_list = list(prot.member)
print('Num interacting proteins:', len(prot_list))

Num interacting proteins: 246


# Calculate p-values and differential expressions

In [10]:
cancer_objects = {'EC':en, 'CO':col, 'LUAD':l}

In [12]:
# Get MAPK pathway proteins
prot = u.get_proteins_in_pathways('MAPK Signaling Pathway', 'wikipathways')
prot_list = list(prot.member)
print('Num interacting proteins:', len(prot_list))

gene = 'KRAS'
merged_dfs = {}
    
for cancer in cancer_objects:
    print(cancer)
    
    # Format df for t-test
    mut_wt = phospho_format_df(cancer_objects[cancer], prot_list)
    mut_wt = mut_wt.dropna(axis = 'columns', how = 'all') #drop cols with only nan
    
    # T-test
    cols_in = list(mut_wt.columns[:-2])
    print('cols tested: ', len(cols_in))
    
    # Get all pvals
    min_num = 5
    all_pval = u.wrap_ttest(mut_wt, 'Mutation', cols_in, return_all = True, correction_method = 'fdr_bh', mincount = min_num)
    all_pval = all_pval.rename(columns = {'Comparison': 'Phospho','P_Value': cancer+'_P_Value'})
    sig = all_pval.loc[all_pval[cancer+'_P_Value'] < 0.05]
    print('Num sig results:', len(sig))
    
    # Get difference in medians
    delta_median_df = get_change_in_medians_df(cancer_objects[cancer], cancer, mut_wt, cols_in)

    # Merge pval_df and delta_median_df
    pval_medians_df = all_pval.merge(delta_median_df, on='Phospho', how='outer')
    pval_medians_df = pval_medians_df.replace(to_replace = r'_phosphoproteomics', value = '', regex = True)
    merged_dfs[cancer] = pval_medians_df # testing purposes
    
    # Create csv
    pval_medians_df.to_csv('csv/'+cancer+'_phospho_MAPK.csv', index=False)
    print(pval_medians_df.head(), '\n')

Num interacting proteins: 246
EC
cols tested:  1375
Num sig results: 17
      Phospho  EC_P_Value  EC_Median
0  MKNK2_S220    0.001348    0.94700
1  NFKB2_S858    0.001348    0.69285
2   RAF1_T330    0.002399    1.12066
3   TAB1_S378    0.002916    0.46350
4  MKNK1_S401    0.003520    0.47800 

CO
cols tested:  729
Num sig results: 0
              Phospho  CO_P_Value  CO_Median
0  TGFBR2_S553_P37173    0.176137     0.4155
1    RELA_S238_Q04206    0.229576    -0.4200
2     NF1_S864_P21359    0.980298     0.0255
3    NF1_S2543_P21359    0.980298     0.2695
4    NF1_S2515_P21359    0.980298    -0.0775 

LUAD
cols tested:  867
Num sig results: 20
          Phospho  LUAD_P_Value  LUAD_Median
0      SOS1_S1161      0.000055      1.30250
1  MKNK1_S209S214      0.015097      1.49000
2      MKNK2_S220      0.015097      1.33870
3       FLNB_S316      0.015097      0.98345
4      FLNB_T2585      0.015097      1.00035 



# Get number of significant genes in each cancer

In [13]:
# Using corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
all_sig = []

for cancer in merged_dfs:
    df = merged_dfs[cancer]
    sig_df = df.loc[df[cancer+'_P_Value'] < 0.1]
    print(cancer, 'sig comparisons:', len(sig_df))
    sig_list = list(sig_df['Phospho'])
    all_sig.append(sig_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = list(set(flat_list)) # keep only one if gene in multiple cancer sig lists
print('\nNumber of significant comparisons in at least 1 cancer:', len(sig))

EC sig comparisons: 26
CO sig comparisons: 0
LUAD sig comparisons: 40

Number of significant comparisons in at least 1 cancer: 62


In [14]:
# Find common sites
e_df = merged_dfs['EC']
e_sig = e_df.loc[e_df['EC_P_Value'] < 0.05]
e_list = list(e_sig.Phospho)

luad_df = merged_dfs['LUAD']
luad_sig = luad_df.loc[luad_df['LUAD_P_Value'] < 0.05]
luad_list = list(luad_sig.Phospho)

both = [value for value in e_list if value in luad_list]
print('Shared sites:', len(both))
both

Shared sites: 3


['MKNK2_S220', 'NFKB2_S858', 'SOS1_S1161']

# Get a list of significant genes in at least one cancer

In [15]:
s = pd.Series(sig)
print('Number of significant genes in at least one cancer:', len(s))
s.to_csv('csv/list_sig_one_cancer_phospho_MAPK.csv', index=False)

Number of significant genes in at least one cancer: 62


# Get a list of significant genes in multiple cancers

In [16]:
# sig in multiple cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant genes in mult cancers:', len(mult))

Number of significant genes in mult cancers: 4


In [17]:
m = pd.Series(mult)
m = m.replace(to_replace = r'_phosphoproteomics$', value = '', regex = True)
m.to_csv('csv/list_sig_mult_cancers_phospho_MAPK.csv', index=False)